In [ ]:
%%capture
!pip install gradio transformers torch

In [ ]:
# import gradio as gr
# import torch
# import json
# import time
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from threading import Timer

# # Khởi tạo model
# tokenizer = AutoTokenizer.from_pretrained("MadeAgents/Hammer2.1-3b")
# model = AutoModelForCausalLM.from_pretrained("MadeAgents/Hammer2.1-3b",
#                                            torch_dtype=torch.bfloat16,
#                                            device_map="auto")

# tools = [
#     {
#         "name": "send_message",
#         "description": "Send a message to a friend, optionally after a delay",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "recipient": {"type": "string", "description": "Username of recipient"},
#                 "message": {"type": "string", "description": "Message content"},
#                 "delay_minutes": {"type": "number", "description": "Delay in minutes before sending"}
#             },
#             "required": ["recipient", "message"]
#         }
#     },
#     {
#         "name": "respond",
#         "description": "Respond to the user",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "message": {"type": "string"}
#             },
#             "required": ["message"]
#         }
#     }
# ]

# def parse_timer_command(message):
#     """Phân tích câu lệnh hẹn giờ"""
#     if "sau" in message and "phút" in message:
#         try:
#             parts = message.split("sau")[1].split("phút")[0].strip()
#             minutes = int(parts)
#             return minutes
#         except:
#             return 0
#     return 0

# def chat_with_ai(message, history):
#     # Tạo messages từ history
#     messages = []
#     for user_msg, ai_msg in history:
#         messages.append({"role": "user", "content": user_msg})
#         if ai_msg:
#             messages.append({"role": "assistant", "content": ai_msg})

#     # Thêm message mới
#     messages.append({"role": "user", "content": message})

#     # Tạo input cho model
#     inputs = tokenizer.apply_chat_template(
#         messages,
#         tools=tools,
#         add_generation_prompt=True,
#         return_dict=True,
#         return_tensors="pt"
#     ).to(model.device)

#     # Generate response
#     outputs = model.generate(**inputs, max_new_tokens=128)
#     response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

#     # Parse tool calls nếu có
#     try:
#         tool_calls = eval(response)  # Lưu ý: chỉ dùng với input đáng tin cậy
#         if isinstance(tool_calls, dict):
#             tool_calls = [tool_calls]

#         if tool_calls and isinstance(tool_calls, list):
#             for call in tool_calls:
#                 if call.get("name") == "send_message":
#                     # Phân tích thời gian hẹn giờ từ message
#                     delay_minutes = parse_timer_command(call.get("arguments", {}).get("message", ""))
#                     call["arguments"]["delay_minutes"] = delay_minutes

#         return json.dumps(tool_calls) if tool_calls else response
#     except:
#         return response

# # Tạo giao diện Gradio
# with gr.Blocks() as demo:
#     gr.Markdown("# 🤖 AI Assistant với Timer")
#     chatbot = gr.Chatbot(height=500)
#     msg = gr.Textbox(label="Nhập tin nhắn")
#     clear = gr.Button("Xóa lịch sử")

#     def respond(message, chat_history):
#         bot_message = chat_with_ai(message, chat_history)
#         chat_history.append((message, bot_message))
#         return "", chat_history

#     msg.submit(respond, [msg, chatbot], [msg, chatbot])
#     clear.click(lambda: None, None, chatbot, queue=False)

# demo.launch(share=True, debug=True)


In [ ]:
import gradio as gr
import torch
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer

# Đảm bảo CUDA khả dụng
device = "cuda" if torch.cuda.is_available() else "cpu"

# Khởi tạo model Hammer và đẩy lên CUDA
hammer_tokenizer = AutoTokenizer.from_pretrained("MadeAgents/Hammer2.1-3b")
hammer_model = AutoModelForCausalLM.from_pretrained(
    "MadeAgents/Hammer2.1-3b",
    torch_dtype=torch.bfloat16
).to(device)

# Khởi tạo model Qwen và đẩy lên CUDA
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
qwen_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16
).to(device)

tools = [
    {
        "name": "send_message",
        "description": "Send a message to a friend, optionally after a delay",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Username of recipient"},
                "message": {"type": "string", "description": "Message content"},
                "delay_minutes": {"type": "number", "description": "Delay in minutes before sending"}
            },
            "required": ["recipient", "message"]
        }
    },
    {
        "name": "respond",
        "description": "Respond to the user",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {"type": "string"}
            },
            "required": ["message"]
        }
    }
]

def parse_timer_command(message):
    """Phân tích câu lệnh hẹn giờ"""
    if "sau" in message and "phút" in message:
        try:
            parts = message.split("sau")[1].split("phút")[0].strip()
            minutes = int(parts)
            return minutes
        except:
            return 0
    return 0

def extract_json_from_response(response):
    """Cố gắng trích xuất JSON từ response"""
    try:
        # Tìm các khối JSON trong response
        json_str = re.search(r'```(?:json)?\n?(.*?)\n?```', response, re.DOTALL)
        if json_str:
            return json.loads(json_str.group(1).strip())

        # Thử parse trực tiếp nếu không có markdown
        return json.loads(response.strip())
    except:
        return None

def chat_with_qwen(messages):
    """Xử lý hội thoại với Qwen khi Hammer không trả về kết quả phù hợp"""
    # Chuẩn bị messages cho Qwen
    qwen_messages = []
    for msg in messages:
        if msg["role"] == "user":
            qwen_messages.append({"role": "user", "content": msg["content"]})
        else:
            qwen_messages.append({"role": "assistant", "content": msg["content"]})

    # Thêm instruction để Qwen tập trung vào hội thoại
    qwen_messages.append({"role": "system", "content": "Bạn là trợ lý ảo hữu ích. Hãy trả lời ngắn gọn, thân thiện."})

    # Generate response
    inputs = qwen_tokenizer.apply_chat_template(
        qwen_messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(qwen_model.device)

    outputs = qwen_model.generate(inputs, max_new_tokens=256)
    response = qwen_tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    return response

def chat_with_ai(message, history):
    # Tạo messages từ history
    messages = []
    for user_msg, ai_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if ai_msg:
            try:
                # Nếu là tool call thì giữ nguyên định dạng
                json.loads(ai_msg)
                messages.append({"role": "assistant", "content": ai_msg, "tool_calls": json.loads(ai_msg)})
            except:
                messages.append({"role": "assistant", "content": ai_msg})

    # Thêm message mới
    messages.append({"role": "user", "content": message})

    # Thử với Hammer trước
    hammer_inputs = hammer_tokenizer.apply_chat_template(
        [{"role": "user", "content": message}],
        tools=tools,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to(hammer_model.device)

    hammer_outputs = hammer_model.generate(**hammer_inputs, max_new_tokens=256)
    hammer_response = hammer_tokenizer.decode(hammer_outputs[0][hammer_inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    # Parse tool calls từ Hammer
    tool_calls = None
    try:
        tool_calls = extract_json_from_response(hammer_response)
        if tool_calls:
            if isinstance(tool_calls, dict):
                tool_calls = [tool_calls]
    except:
        pass

    # Xử lý kết quả từ Hammer
    if tool_calls and isinstance(tool_calls, list):
        processed_calls = []
        for call in tool_calls:
            if isinstance(call, dict) and call.get("name") == "send_message":
                # Thêm thông báo gửi tin nhắn vào history
                messages.append({
                    "role": "assistant",
                    "content": f"Tôi vừa thực hiện gửi tin nhắn cho người {call.get('arguments', {}).get('recipient', '')}"
                })

                # Xử lý delay
                delay_minutes = parse_timer_command(call.get("arguments", {}).get("message", ""))
                if isinstance(call.get("arguments"), dict):
                    call["arguments"]["delay_minutes"] = delay_minutes

                processed_calls.append(call)

        if processed_calls:
            return json.dumps(processed_calls, ensure_ascii=False)

    # Nếu Hammer không trả về tool call hợp lệ, chuyển sang Qwen
    qwen_response = chat_with_qwen(messages)
    return qwen_response

# Tạo giao diện Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 AI Assistant với Hammer và Qwen")
    chatbot = gr.Chatbot(height=500)
    msg = gr.Textbox(label="Nhập tin nhắn")
    clear = gr.Button("Xóa lịch sử")

    def respond(message, chat_history):
        bot_message = chat_with_ai(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

<ipython-input-3-b06d0efd90b8>:166: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a45ffc66e649db6c60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
